In [1]:
import pandas as pd

df = pd.read_csv('/content/movie_data.csv')
df.head(10)

,review,sentiment
0,"In 1974, the teenager Martha Moxley (Maggie Gr...",1
1,OK... so... I really like Kris Kristofferson a...,0
2,"***SPOILER*** Do not read this, if you think a...",0
3,hi for all the people who have seen this wonde...,1
4,"I recently bought the DVD, forgetting just how...",0
5,Leave it to Braik to put on a good show. Final...,1
6,Nathan Detroit (Frank Sinatra) is the manager ...,1
7,"To understand ""Crash Course"" in the right cont...",1
8,I've been impressed with Chavez's stance again...,1
9,This movie is directed by Renny Harlin the fin...,1


In [2]:
df['review'][0]

'In 1974, the teenager Martha Moxley (Maggie Grace) moves to the high-class area of Belle Haven, Greenwich, Connecticut. On the Mischief Night, eve of Halloween, she was murdered in the backyard of her house and her murder remained unsolved. Twenty-two years later, the writer Mark Fuhrman (Christopher Meloni), who is a former LA detective that has fallen in disgrace for perjury in O.J. Simpson trial and moved to Idaho, decides to investigate the case with his partner Stephen Weeks (Andrew Mitchell) with the purpose of writing a book. The locals squirm and do not welcome them, but with the support of the retired detective Steve Carroll (Robert Forster) that was in charge of the investigation in the 70\'s, they discover the criminal and a net of power and money to cover the murder.<br /><br />"Murder in Greenwich" is a good TV movie, with the true story of a murder of a fifteen years old girl that was committed by a wealthy teenager whose mother was a Kennedy. The powerful and rich famil

In [3]:
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer

count = CountVectorizer()


docs = np.array(['Roses are red',
                'Violets are too',
                'I’m colorblind, What about you?'])

bag = count.fit_transform(docs)

In [4]:
print(count.vocabulary_)

{'roses': 4, 'are': 1, 'red': 3, 'violets': 6, 'too': 5, 'colorblind': 2, 'what': 7, 'about': 0, 'you': 8}


In [5]:
print(bag.toarray())

[[0 1 0 1 1 0 0 0 0]
 [0 1 0 0 0 1 1 0 0]
 [1 0 1 0 0 0 0 1 1]]


In [6]:
# Word relevancy using term frequency-inverse document frequency

from sklearn.feature_extraction.text import TfidfTransformer
np.set_printoptions(precision=2)

tfidf = TfidfTransformer(use_idf=True, norm='l2', smooth_idf=True)
print(tfidf.fit_transform(count.fit_transform(docs)).toarray())

[[0.   0.47 0.   0.62 0.62 0.   0.   0.   0.  ]
 [0.   0.47 0.   0.   0.   0.62 0.62 0.   0.  ]
 [0.5  0.   0.5  0.   0.   0.   0.   0.5  0.5 ]]


In [7]:
#Data preparation
import pandas as pd

df = pd.read_csv('/content/movie_data.csv')
df.loc[0, 'review'][-50:]

'is seven.<br /><br />Title (Brazil): Not Available'

In [8]:
import re
def preprocessor(text):
    text = re.sub('<[^>]*>', '', text)
    emotions = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)', text)
    text = re.sub('[\W]+', ' ', text.lower()) +\
        ' '.join(emotions).replace('-', '')
    return text

In [9]:
#testing whether the cleaning works
preprocessor(df.loc[0, 'review'][-50:])

'is seven title brazil not available'

In [10]:
#testing emojis
preprocessor("</a>This :) is a :( test :-)!")

'this is a test :) :( :)'

In [11]:
df['review'] = df['review'].apply(preprocessor)

In [12]:
#tokenization of documents
from nltk.stem.porter import PorterStemmer

porter=PorterStemmer()

In [13]:
def tokenizer(text):
    return text.split()

In [14]:
def tokenizer_porter(text):
    return [porter.stem(word) for word in text.split()]

In [15]:
tokenizer('Akaka is a diligent and hardworking boy')

['Akaka', 'is', 'a', 'diligent', 'and', 'hardworking', 'boy']

In [16]:
tokenizer_porter('Akaka is a diligent and hardworking boy')

['akaka', 'is', 'a', 'dilig', 'and', 'hardwork', 'boy']

In [17]:
#removes articles
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [18]:
from nltk.corpus import stopwords

stop=stopwords.words('english')
[w for w in tokenizer_porter('an Akaka is a diligent and hardworking boy and reads alot') [-10:] if w not in stop]

['akaka', 'dilig', 'hardwork', 'boy', 'read', 'alot']

In [20]:
# Transforming text data into TF-IDF vectors
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(strip_accents=None,
                       lowercase=False,
                       preprocessor=None,
                       tokenizer=tokenizer_porter,
                       use_idf=True,
                       norm='l2',
                       smooth_idf=True)

y = df.sentiment.values
x = tfidf.fit_transform(df.review)

In [21]:
#Classifying the document using Logistic regression
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x, y, random_state=1, test_size=0.5,
                                                   shuffle=False)

In [22]:
import pickle
from sklearn.linear_model import LogisticRegressionCV

clf = LogisticRegressionCV(cv=5,
                          scoring='accuracy',
                          random_state=0,
                          n_jobs=-1,
                          verbose=3,
                          max_iter=300).fit(x_train, y_train)
saved_model = open('saved_model.sav', 'wb')
pickle.dump(clf, saved_model)
saved_model.close()

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:  2.9min finished


In [23]:
#modelevaluation
filename = 'saved_model.sav'
saved_clf = pickle.load(open(filename, 'rb'))

In [24]:
saved_clf.score(x_test, y_test)

0.89608

In [25]:
#Test the model by trying to predict
pred=saved_clf.predict(x_test)

In [26]:
print(pred)

[0 1 1 ... 0 1 1]
